In [1]:
import subprocess
import sys

try:
    import memory_profiler
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", 'memory-profiler'])
finally:
    import memory_profiler

from memory_profiler import memory_usage
import pytest
import time
import os.path
import re
import inspect
from typing import List
import time

In [103]:
# KINDLY GO THROUGH TEST FILE TO UNDERSTAND
from typing import List
import time

# Here in this code we will be leaking memory because we are creating cyclic reference.
# Find that we are indeed making cyclic references.
# Eventually memory will be released, but that is currently not happening immediately.
# We have added a function called "clear_memory" but it is not able to do it's job. Fix it.
# Refer to test_clear_memory Test in test_session2.py to see how we're crudely finding that
# this code is sub-optimal.
class Something(object):

    def __init__(self,i):
        super().__init__()
        self.something_new = SomethingNew(i, self)
        #print(f'something self: {hex(id(self))}, something.something_new: {hex(id(self.something_new))}')

class SomethingNew(object):

    def __init__(self, i: int = 0, something: Something = None):
        super().__init__()
        self.i = i
        self.something = something
        #print(f'something_new self{hex(id(self))}, something_new.something: {hex(id(self.something))}, something_new.i: {hex(id(self.i))}')


def add_something(collection: List[Something], i: int):
    my_var = Something(i)
    #something.something_new = SomethingNew(i, something)
    collection.append(my_var)

def reserved_function():
    # to be used in future if required
    pass

def clear_memory(collection: List[Something]):
    # you probably need to add some comment here
    #print("1 collection", collection)
    for item in collection:
        del item.something_new
    collection.clear()
    #print("2",collection)
    #gc.collect()


def critical_function():
    collection = list()
    for i in range(1, 1024 * 128):
    #for i in range(1, 4):
        add_something(collection, i)
    #print(len(collection))
    clear_memory(collection)

In [24]:
# Here we are suboptimally testing whether two strings are exactly same or not
# After that we are trying to see if we have a particular character in that string or not
# Currently the code is suboptimal. Write it in such a way that it takes 1/10 the current time

# DO NOT CHANGE THIS PROGRAM
def compare_strings_old(n):
    a = 'a long string that is not intered' * 200
    b = 'a long string that is not intered' * 200
    for i in range(n):
        if a == b:
            pass
    char_list = list(a)
    for i in range(n):
        if 'd' in char_list:
            pass

# Trial 1 Removing the for loop
'''
def compare_strings_new(n):
    a = 'a long string that is not intered' * 200
    b = 'a long string that is not intered' * 200
    #for i in range(n):
    if a == b:
        #print("True")
        pass
    #char_list = list(a)
    #for i in range(n):
    if'd' in a:
        pass
    #time.sleep(6) # remove this line, this is just to simulate your "slow" code
'''
# Trial 2 Using sys.itern 
def compare_strings_new(n):
    a = sys.intern('a long string that is not intered' * 200)
    b = sys.intern('a long string that is not intered' * 200)
    char_set = set(a)
    for i in range(n):
        if a is b:
            pass
        if 'd' in char_set:
            pass

In [25]:
def test_performance():
    start1 = time.perf_counter()
    print("start1", start1)
    compare_strings_old(10000000)
    end1 = time.perf_counter()
    delta1 = end1 - start1
    print("del1", delta1)

    start2 = time.perf_counter()
    compare_strings_new(10000000)
    end2 = time.perf_counter()
    delta2 = end2 - start2
    print("del2", delta2)
    
    print(delta1/delta2)

    assert delta1 / delta2 >= 10.0

In [26]:
test_performance()
#

start1 1.7e-06
del1 91.7911313
del2 4.861266999999998
18.88214148698272


In [105]:
memory_used = []
def test_clear_memory():
    memory_used = memory_usage((critical_function))
    print("len(m/m used), max(memory_used)", len(memory_used), max(memory_used))
    #print((memory_used[len(memory_used) - 1] - memory_used[0]))
    assert (memory_used[len(memory_used) - 1] - memory_used[0]) < 4


def test_memory_actually_increased():
    # This test will check whether we are actually increase the memory during running the function f
    memory_used2 = memory_usage((critical_function))
    print("len(m/m used2), max(memory_used2)", len(memory_used2), max(memory_used2))
    peak = max(memory_used2)
   # print(peak - memory_used2[0])
    assert (peak - memory_used2[0]) > 8, "Seems like you have changed the program! Are you trying to cheat!"

In [108]:
help(critical_function())

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).



In [107]:
test_clear_memory()
test_memory_actually_increased()

len(m/m used), max(memory_used) 38 107.4296875
len(m/m used2), max(memory_used2) 14 107.43359375


In [33]:
class Something(object):

    def __init__(self):
        super().__init__()
        self.something_new = SomethingNew(1, self)
        print(f'something self: {hex(id(self))}, something.something_new: {hex(id(self.something_new))}')


class SomethingNew(object):

    def __init__(self, i: int = 0, something: Something = None):
        super().__init__()
        self.i = i
        self.something = something
        print(f'something_new self{hex(id(self))}, something_new.something: {hex(id(self.something))}, something_new.i: {hex(id(self.i))}')
        #print(self.i)

In [34]:
something = Something()
#something.something_new = SomethingNew(1, something)

something_new self0x26c79daecf8, something_new.something: 0x26c79daeeb8, something_new.i: 0x63d56c40
something self: 0x26c79daeeb8, something.something_new: 0x26c79daecf8


In [116]:
a = "prema"
if 'p' in a:
    pass

In [17]:
class Something(object):
    def __init__(self):
        super().__init__()
        self.something_new = None
        # print(f'something self: {hex(id(self))}, something.something_new: {hex(id(self.something_new))}')
        # Print Function used to assert indeed an cyclic reference is created

class SomethingNew(object):
    def __init__(self, i: int = 0, something: Something = None):
        super().__init__()
        self.i = i
        self.something = something
        # print(f'something_new self{hex(id(self))}, something_new.something: {hex(id(self.something))}, something_new.i: {hex(id(self.i))}')
        # Print Function used to assert indeed an cyclic reference is created

In [21]:
def add_something(collection: List[Something], i: int):
    something = Something()
    something.something_new = SomethingNew(i,something)
    collection.append(something)
def reserved_function():
    # to be used in future if required
    pass

def clear_memory(collection: List[Something]):
    # you probably need to add some comment here
    #print("1 collection", collection)
    for item in collection:
        del item.something_new
    collection.clear()
    #print("2",collection)
    #gc.collect()


def critical_function():
    collection = list()
    for i in range(1, 1024 * 128):
    #for i in range(1, 4):
        add_something(collection, i)
    #print(len(collection))
    clear_memory(collection)

In [22]:
memory_used = []
def test_clear_memory():
    memory_used = memory_usage((critical_function))
    print("len(m/m used), max(memory_used)", len(memory_used), max(memory_used))
    #print((memory_used[len(memory_used) - 1] - memory_used[0]))
    assert (memory_used[len(memory_used) - 1] - memory_used[0]) < 4


def test_memory_actually_increased():
    # This test will check whether we are actually increase the memory during running the function f
    memory_used2 = memory_usage((critical_function))
    print("len(m/m used2), max(memory_used2)", len(memory_used2), max(memory_used2))
    peak = max(memory_used2)
   # print(peak - memory_used2[0])
    assert (peak - memory_used2[0]) > 8, "Seems like you have changed the program! Are you trying to cheat!"

In [23]:
test_clear_memory()
test_memory_actually_increased()

len(m/m used), max(memory_used) 29 104.34765625
len(m/m used2), max(memory_used2) 50 117.17578125


In [27]:
# KINDLY GO THROUGH TEST FILE TO UNDERSTAND
from typing import List
import time
import sys

# Here in this code we will be leaking memory because we are creating cyclic reference.
# Find that we are indeed making cyclic references.
# Eventually memory will be released, but that is currently not happening immediately.
# We have added a function called "clear_memory" but it is not able to do it's job. Fix it.
# Refer to test_clear_memory Test in test_session2.py to see how we're crudely finding that
# this code is sub-optimal.
class Something(object):
    def __init__(self):
        super().__init__()
        self.something_new = None
        # print(f'something self: {hex(id(self))}, something.something_new: {hex(id(self.something_new))}')
        # Print Function used to assert indeed an cyclic reference is created

class SomethingNew(object):
    def __init__(self, i: int = 0, something: Something = None):
        super().__init__()
        self.i = i
        self.something = something
        # print(f'something_new self{hex(id(self))}, something_new.something: {hex(id(self.something))}, something_new.i: {hex(id(self.i))}')
        # Print Function used to assert indeed an cyclic reference is created

def add_something(collection: List[Something], i: int):
    something = Something()
    something.something_new = SomethingNew(i,something)
    collection.append(something)

def reserved_function():
    # to be used in future if required
    pass

def clear_memory(collection: List[Something]):
    # clearing the memory of the cyclic object instead of just the list elements
    for item in collection:
        del item.something_new
    collection.clear()


def critical_function():
    collection = list()
    for i in range(1, 1024 * 128):
        add_something(collection, i)
    clear_memory(collection)


# Here we are suboptimally testing whether two strings are exactly same or not
# After that we are trying to see if we have a particular character in that string or not
# Currently the code is suboptimal. Write it in such a way that it takes 1/10 the current time

# DO NOT CHANGE THIS PROGRAM
def compare_strings_old(n):
    a = 'a long string that is not intered' * 200
    b = 'a long string that is not intered' * 200
    for i in range(n):
        if a == b:
            pass
    char_list = list(a)
    for i in range(n):
        if 'd' in char_list:
            pass

# YOU NEED TO CHANGE THIS PROGRAM
def compare_strings_new(n):
    a = sys.intern('a long string that is not intered' * 200)
    b = sys.intern('a long string that is not intered' * 200)
    char_set = set(a)
    for i in range(n):
        if a is b:
            pass
        if 'd' in char_set:
            pass

In [36]:
def test_class_repr():
    s = Something()
    s_n = SomethingNew()
    print(s.__repr__())
    print(s_n.__repr__())
    print('object at' not in s.__repr__())
    assert 'object at' not in s.__repr__() and 'object at' not in s_n.__repr__()

In [37]:
test_class_repr()

False


AssertionError: 

In [35]:
'object at' not in s.__repr__()

NameError: name 's' is not defined